In [1]:
import pandas as pd
import numpy as np
import trading_calendar as tc

## Part 1: Creating a MultiIndex Object (Multi-index DataFrame)

### Loading The Data

In [2]:
df = pd.read_csv("prices.csv", index_col=0)
df.head()

,TSLA,MSFT,GE
date,,,
2019-01-02,310.12,99.6466,7.7134
2019-01-03,300.36,95.9808,7.7229
2019-01-04,317.69,100.4448,7.8858
2019-01-07,334.96,100.5729,8.3745
2019-01-08,335.35,101.3021,8.2020


### Stacking

In [3]:
df.stack()

date            
2019-01-02  TSLA    310.1200
            MSFT     99.6466
            GE        7.7134
2019-01-03  TSLA    300.3600
            MSFT     95.9808
                      ...   
2019-10-30  MSFT    144.1196
            GE       10.1100
2019-10-31  TSLA    314.9200
            MSFT    142.8838
            GE        9.9800
Length: 633, dtype: float64

Stacking returns a Series:

In [4]:
type(df.stack())

pandas.core.series.Series

### Convert to DataFrame

In [5]:
df.stack().to_frame(name='price')

price
date                     
2019-01-02 TSLA  310.1200
           MSFT   99.6466
           GE      7.7134
2019-01-03 TSLA  300.3600
           MSFT   95.9808
...                   ...
2019-10-30 MSFT  144.1196
           GE     10.1100
2019-10-31 TSLA  314.9200
           MSFT  142.8838
           GE      9.9800

[633 rows x 1 columns]

In [6]:
type(df.stack().to_frame(name='price'))

pandas.core.frame.DataFrame

### The Function to Convert a DataFrame to a Multi-index DataFrame

Use this function to convert a df to an mdf. Include the `trading_calendar.py` script if you'd like to get proper market-open dates.

In [7]:
def df2mdf(df, trading_day='C'):
    mdf = df.stack().to_frame(name='price')
    mdf.index.names = [None, None]
    new_dates = np.unique(mdf.apply(lambda x: pd.Timestamp(x.name[0], tz='UTC', freq=trading_day), axis=1).values)
    mdf.index = mdf.index.set_levels(new_dates, level=0)
    return mdf

mdf = df2mdf(df, trading_day=tc.trading_day)
mdf.head()

price
2019-01-02 TSLA  310.1200
           MSFT   99.6466
           GE      7.7134
2019-01-03 TSLA  300.3600
           MSFT   95.9808

In [9]:
type(mdf)

pandas.core.frame.DataFrame

In [10]:
mdf

price
2019-01-02 TSLA  310.1200
           MSFT   99.6466
           GE      7.7134
2019-01-03 TSLA  300.3600
           MSFT   95.9808
...                   ...
2019-10-30 MSFT  144.1196
           GE     10.1100
2019-10-31 TSLA  314.9200
           MSFT  142.8838
           GE      9.9800

[633 rows x 1 columns]

In [11]:
mdf.iloc[0]

price    310.12
Name: (2019-01-02 00:00:00, TSLA), dtype: float64

## Part 2. MultiIndex Operations

### Select by index

In [12]:
mdf[mdf.index.get_level_values(1) == 'TSLA']

,,price
2019-01-02,TSLA,310.12
2019-01-03,TSLA,300.36
2019-01-04,TSLA,317.69
2019-01-07,TSLA,334.96
2019-01-08,TSLA,335.35
...,...,...
2019-10-25,TSLA,328.13
2019-10-28,TSLA,327.71
2019-10-29,TSLA,316.22
2019-10-30,TSLA,315.01


### Select by multiple symbols

In [13]:
m = mdf.index.get_level_values(1).isin(['TSLA', 'GE'])
mdf[m]

price
2019-01-02 TSLA  310.1200
           GE      7.7134
2019-01-03 TSLA  300.3600
           GE      7.7229
2019-01-04 TSLA  317.6900
...                   ...
2019-10-29 GE      9.0700
2019-10-30 TSLA  315.0100
           GE     10.1100
2019-10-31 TSLA  314.9200
           GE      9.9800

[422 rows x 1 columns]

### Select by date (index level 0)

Index level 0 can also be accessed directly with the loc selector. A date object can quickly be referenced with a string, which is really convenient.

In [14]:
mdf.loc['2019-01-02']

price
2019-01-02 TSLA  310.1200
           MSFT   99.6466
           GE      7.7134

### Select by date range

In [15]:
mdf.loc['2019-01-02':'2019-01-07']

price
2019-01-02 TSLA  310.1200
           MSFT   99.6466
           GE      7.7134
2019-01-03 TSLA  300.3600
           MSFT   95.9808
           GE      7.7229
2019-01-04 TSLA  317.6900
           MSFT  100.4448
           GE      7.8858
2019-01-07 TSLA  334.9600
           MSFT  100.5729
           GE      8.3745

### Select by row numbers

In [16]:
mdf.iloc[0]

price    310.12
Name: (2019-01-02 00:00:00, TSLA), dtype: float64

### Select by a single index pair

Can be done with loc:

In [17]:
mdf.loc[('2019-01-02', 'TSLA')]

price    310.12
Name: (2019-01-02 00:00:00, TSLA), dtype: float64

Or cross-sectional selection (this is less ideal since you can't see the date index, so avoid this!):

In [18]:
mdf.xs('2019-01-02').xs('TSLA')

price    310.12
Name: TSLA, dtype: float64

### Select by multiple indexes
The easiest way to do this is by chaining the selections e.g. select by level 0 index first, followed by level 1 index.

In [19]:
mdf1 = mdf.loc['2019-01-02':'2019-01-07']
mdf1[mdf1.index.get_level_values(1).isin(['MSFT', 'GE'])]

price
2019-01-02 MSFT   99.6466
           GE      7.7134
2019-01-03 MSFT   95.9808
           GE      7.7229
2019-01-04 MSFT  100.4448
           GE      7.8858
2019-01-07 MSFT  100.5729
           GE      8.3745